<a href="https://colab.research.google.com/github/HWANG-HOE-SUN/Handson/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 기본 데이터 정리 및 처리
import numpy as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')
import missingno

# 전처리 및 머신 러닝 알고리즘
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

# 모델 튜닝 및 평가
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn import model_selection

In [2]:
# 경고 제거(판다스 error 메시지 무시)
import sys
import warnings

warnings.filterwarnings('ignore')

#핸즈온 Chapter 10 인공신경망

In [3]:
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

In [4]:
# 퍼셉트론 학습 알고리즘 

iris = load_iris()
X = iris.data[:,(2,3)] # 꽃잎 길이, 너비
y = (iris.target == 0 ).astype(np.int) # Setosa인가? = 이중분류

per_clf = Perceptron()
per_clf.fit(X,y)
X_
y_pred = per_clf.predict([[2, 0.5]])

In [5]:
# https://skyil.tistory.com/68  학습 관련 종결
# 미니배치 확률경사하강법(MSGD) 
# ※배치?BGD 한 에포크(실험)당 전체Data를 통해 gradient학습 
  # 미니배치 MGD 한 에포크(실험)당 일부(Mini-Batch)데이터 셋을 통해 gradient학습
  # 확률적 경사하강 SGD 한 에포크(실험)당 한 개의(랜덤추출) Data를 통해 gradient학습 (데이터 사이즈n만큼의 epoch가능)
  # 일반적으로, MGD + SGD인 MSGD를 사용. Batch size를 결정하고( 전체 1000개라면 100개의 batch size!로 10묶음, 즉 10epoch까지 가능)
                                       # 확률적으로 gradient학습을 하여서, global optimal에 가깝게 답을 찾아간다. (전체에 나누어 떨어지게 batch_size정해야 데이터 고려에 있어서 공평함)

In [10]:
import tensorflow as tf
from tensorflow import keras
tf.__version__, keras.__version__

('2.4.0', '2.4.0')

In [11]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [14]:
X_train_full.shape, X_train_full.dtype

((60000, 28, 28), dtype('uint8'))

In [16]:
# 검증세트 와 훈련세트로 나누겠음
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0 #실수로 자동형변환을 위해 255.0으로 나눔
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

In [17]:
#레이블의 직관적 이해(어느 옷을 나타내는건지)를 위해 클래스 이름을 지정하겠음
class_names = ['T-shirt/Top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] 

In [19]:
class_names[y_train[0]]

'Coat'

In [21]:
# 두개의 은닉층 사용, 다층 퍼셉트론
model = keras.models.Sequential() # 가장 간단한 신경망 모델. 순서대로 연결된 층을 일렬로 쌓아서 구성한다( 시퀀셜 API라고 칭함 ) 
model.add(keras.layers.Flatten(input_shape=[28,28])) # 첫번째 층을 모델에 추가해준다. Flatten은 이미지를 1D배열로 변환함 즉, X.reshape(-1,1)의 효과  
# 활성화 함수 다수
model.add(keras.layers.Dense(300,activation="relu")) # 뉴런 300개인 Dense은닉층. 이 층에선 Relu를 쓰겠다. Dense층마다 각자 가중치 행렬을 관리함.
model.add(keras.layers.Dense(100,activation="relu")) # 뉴런 100개인 Dense은닉층. 
model.add(keras.layers.Dense(10,activation='softmax')) # (클래스, 즉 옷종류마다 하나씩) 뉴런 10개를 가진 Dense출력층. 배타적인 클래스이므로, 소프트맥스 활성화 함수를 사용함.

In [23]:
# 위의 코드와 같은 효과. 층을 리스트로 그냥 전달할 수도 있다.
model = keras.models.Sequential(
    [
    keras.layers.Flatten(input_shape=[28,28]),   
    keras.layers.Dense(300,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(10,activation='softmax') 
    ]
) 

In [24]:
model.summary()  # 이 모델에선 총 266,610의 파라미터 사용. 모두 훈련되는(Trainable) 파라미터다.

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________
